# Linked Art - Visualisation - John Ruskin

This notebook is concerned with a visualisation of artworks represented in Linked Art.


## Import What We Need

See the [Transform Introduction](01-00-Transform-Intro.ipynb) for a description of the Python libraries used

In [41]:
try:
    import ipywidgets as widgets
except:
    !pip install ipywidgets
    import ipywidgets as widgets

from ipywidgets import Layout
from ipywidgets import FileUpload

import os


try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display, IFrame, HTML

try:
    import xmltodict
except:
    !pip install xmltodict
    import xmltodict

try:
    import json
except:
    !pip install json
    import json 
    
    
try:
    import requests
except:
    !pip install requests
    import requests

import csv

try:
    import cromulent 
except:
    !pip install cromulent
    import cromulent
    
from cromulent.model import factory
from lib import linkedart as la


try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd


In [42]:
def fn():       # 1.Get file names from directory
    file_list=os.listdir(r"./data/ruskin/output/json")
   
    for file in file_list:
        display(HTML("<a target='_new' href='./data/ruskin/output/json/" + file +"'>" + file + "</a>"))


outputdir = "./data/ruskin/output/json"


images = {  
    "https://www.nga.gov/collection/70238" : "https://media.nga.gov/iiif/49a6128c-8d5a-4b00-beb3-36b29b97c0a1/full/!384,384/0/default.jpg", # tower
    "https://www.nga.gov/collection/70367" : "https://media.nga.gov/iiif/9310c903-7099-4138-b380-d90af8b06703/full/!384,384/0/default.jpg", #tree study
    "https://www.nga.gov/collection/72870" : "https://media.nga.gov/iiif/f6ef48d3-3512-4f46-ac5b-c221b3fa320e/full/!384,384/0/default.jpg" , # florence
    "https://www.nga.gov/collection/76140": "https://media.nga.gov/iiif/bc567179-9c1e-4493-b712-956cc4e6b00a/full/!384,384/0/default.jpg", # stones
    "https://clevelandart.org/art/154494": "https://piction.clevelandart.org/cma/ump.di?e=09E1858D0AB6939812E58428D451CA97E557EAA1228FEB07BD281BBBD4FC3A89&s=24247294&se=1946452765&v=4&f=1989.14_o4.jpg", # sycamore
    "https://www.philamuseum.org/collection/object/pma2" : "https://iiif.micr.io/YwKQj/full/1200,/0/default.jpg",
    "https://www.tate.org.uk/art/artworks/13031" : "http://www.tate.org.uk/art/images/work/N/N02/N02726_8.jpg",
     "https://www.tate.org.uk/art/artworks/13032" : "http://www.tate.org.uk/art/images/work/N/N02/N02972_8.jpg",
    "https://www.tate.org.uk/art/artworks/13033" : "http://www.tate.org.uk/art/images/work/N/N03/N03507_8.jpg"
}


### National Gallery of Art

- [data file](./data/nga/input/nga_ruskin.csv)

In [43]:

#  baseURI for JSON-LD document
baseURI = "https://www.nga.gov/collection/"
file = 'data/nga/input/nga_ruskin.csv'
mpg = pd.read_csv(file)
mpg.head()

,objectid,accessioned,accessionnum,locationid,title,displaydate,beginyear,endyear,visualbrowsertimespan,medium,...,visualbrowserclassification,parentid,isvirtual,departmentabbr,portfolio,series,volume,watermarks,lastdetectedmodification,customprinturl
0,70238,1,1987.73.2,NaN,Tower of the Cathedral at Sens,c. 1845,1845,1845,1826 to 1850,"pen and brown ink, brush and black ink, black ...",...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN
1,70367,1,1988.20.38,NaN,Tree Study,mid-1850s,1845,1855,1826 to 1850,pen and black ink with blue-gray and gray wash...,...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2020-04-10 22:01:40.093-04,NaN
2,72870,1,1991.88.1,NaN,The Garden of San Miniato near Florence,1845,1845,1845,1826 to 1850,"watercolor and pen and black ink, heightened w...",...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN
3,76140,1,1995.52.158,NaN,"Ornamental Study with Acanthus Motif for ""The ...",1849,1849,1849,1826 to 1850,pen and brown ink with watercolor and graphite...,...,drawing,NaN,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN


In [44]:


#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

mapp_nga =  {
    "id":"objectid",
    "accession_number":"accessionnum",
    "accession_date": "",
    "classification" : "classification",
    "title": "title",
    "alt_title": "",
    "notes": "",
    "date_created":"displaydate",
    "date_created_earliest": "beginyear",
    "date_created_latest": "endyear",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "created_provenance" : "",
    "creator":"attribution",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "dimensions",
    "created_provenance": "" ,
    "credit_line": "creditline",
    "collection" : "departmentabbr",
    "current_status" : "",
    "current_owner" : "",
    "homepage": ""
}

def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = [{"name": obj[key]}]
                else:
                    objProp[prop] = obj[key]
    return objProp   

    
allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp_nga)
    id = objProp["id"]
    objProp["homepage"] = "https://www.nga.gov/collection/art-object-page." + id + ".html"
    object_uri = baseURI + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)

    # write to file 
    text_file = open(outputdir + "/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()

### Cleveland Museum of Art 
http://www.clevelandart.org/
- Download CSV file from GitHub https://github.com/ClevelandMuseumArt/openaccess
- View file with OpenRefine https://openrefine.org/
- Create a text facet for the Creator field to identify artworks by John Ruskin
- Export this subset of records (1 record found)
- [data file](./data/cma/input/ruskin.csv) 


In [45]:
#  baseURI for JSON-LD document
baseURI = "https://clevelandart.org/art/"
file = './data/cma/input/ruskin.csv'
mpg = pd.read_csv(file)
mpg.head()

,id,accession_number,share_license_status,tombstone,current_location,title,title_in_original_language,series,series_in_original_language,creation_date,...,digital_description,wall_description,external_resources,citations,catalogue_raisonne,url,image_web,image_print,image_full,updated_at
0,154494,1989.14,CC0,"Budding Sycamore, c. 1876. John Ruskin (Britis...",NaN,Budding Sycamore,NaN,NaN,NaN,c. 1876,...,The most influential art critic in Britain dur...,The most influential art critic in Britain dur...,"{'wikidata': [], 'internet_archive': []}","Cook, Edward Tyas and Alexander Wedderburn. <e...",NaN,https://clevelandart.org/art/1989.14,https://openaccess-cdn.clevelandart.org/1989.1...,https://openaccess-cdn.clevelandart.org/1989.1...,https://openaccess-cdn.clevelandart.org/1989.1...,2021-03-27 12:16:10.046000


In [46]:

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

mapp_cma =  {
    "id":"id",
    "accession_number":"accession_number",
    "accession_date": "",
    "classification" : "type",
    "title": "title",
    "alt_title": "title_in_original_language",
    "notes": "tombstone",
    "date_created":"creation_date",
    "date_created_earliest": "creation_date_earliest",
    "date_created_latest": "creation_date_latest",
    "created_period":"culture",
    "created_dynasty":"",
    "created_inscriptions":"inscriptions",
    "created_notes": "fun_fact",
    "creator":"creator",
    "physical_medium": "Medium",
    "physical_style": "",
    "physical_technique": "technique",
    "physical_description": "",
    "physical_dimensions": "measurements",
    "created_provenance": "provenance" ,
    "credit_line": "creditline",
    "collection" : "department",
    "current_status" : "current_location",
    "current_owner": "",
    "creator": {"name": "John Ruskin"},
    "homepage": "url"
    
}


allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp_cma)

    id = obj[mapp_cma.get("id")]
    object_uri = baseURI + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
    # write to file 
    text_file = open(outputdir + "/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()

### Philadelphia Museum of Art

- [data file](./data/pma/input/pma_ruskin.csv)

In [47]:
baseURI = "https://www.philamuseum.org/collection/object/"

file = './data/pma/input/pma_ruskin.csv'
mpg = pd.read_csv(file)
mpg.head()

,Media,Object Number,Department,Classification,Culture,Period,Display Name,Object Name,Title,Dated,Medium,Dimensions,Description,Attribution,AttributionSort,Credit Line
0,0,64-1993-1,PDP,Drawings,NaN,NaN,John Ruskin,Drawings,"Study of a River Bank, Beauvais, France",1846,Watercolor,H: 125mm W: 175mm,NaN,John Ruskin,Ruskin John,NaN
1,1,01/07/1995,PDP,Drawings,NaN,NaN,John Ruskin,Drawings,Beanstalk,Date unknown,"Pen and yellow-brown ink and wash, graphite pe...",Sheet: 7 3/4 x 11 1/8inches (19.7 x 28.3cm),NaN,John Ruskin,Ruskin John,Purchased with The Herbert & Nannette Rothschi...


In [48]:

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

mapp_pma =  {
    "id":"Object Number",
    "accession_number":"Object Number",
    "accession_date": "",
    "classification" : "Classification",
    "title": "Title",
    "alt_title": "",
    "notes": "",
    "date_created":"Dated",
    "date_created_earliest": "Dated",
    "date_created_latest": "Dated",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"Attribution",
    "physical_medium": "Medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "",
    "created_provenance": "" ,
    "credit_line": "Credit Line",
    "collection" : "Department",
    "current_status" : "",
    "current_owner": "",
    "homepage": "Object Number"
}


allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

cnt=0
for obj in allObjects:
    
    cnt=cnt+1
    # create object property dictionary
    objProp = createObjProp(obj,mapp_pma)
    objProp["creator"] = [{"id": "","name": "John Ruskin"}]
    
    id = "pma" + str(cnt)
    object_uri = baseURI + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
     
    # write to file 
    text_file = open(outputdir + "/" + id + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()

## Tate Museum

The Tate Museum has three items by John Ruskin in its Collection Online database. JSON format file are available from 

https://github.com/tategallery/collection

### N02726 

https://www.tate.org.uk/art/artworks/ruskin-an-olive-spray-and-two-leaf-outlines-n02726 

https://raw.githubusercontent.com/tategallery/collection/master/artworks/n/027/n02726-13031.json 

 

### N02972 

https://www.tate.org.uk/art/artworks/ruskin-the-north-west-angle-of-the-facade-of-st-marks-venice-n02972 

https://raw.githubusercontent.com/tategallery/collection/master/artworks/n/029/n02972-13032.json  

 
### N03507 

https://www.tate.org.uk/art/artworks/ruskin-view-of-bologna-n03507 

https://raw.githubusercontent.com/tategallery/collection/master/artworks/n/035/n03507-13033.json  

In [49]:
fileprefix = "./data/tate/"

files = ("n02726-13031.json", 
         "n02972-13032.json", 
         "n03507-13033.json")

baseURI = "https://www.tate.org.uk/art/artworks/"

mapp_tate =  {
    "id":"id",
    "accession_number":"acno",
    "accession_date": "acquisitionYear",
    "classification" : "classification",
    "title": "title",
    "alt_title": "",
    "notes": "",
    "date_created":"dateText",
    "date_created_earliest": "Dated",
    "date_created_latest": "Dated",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"contributors",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "dimensions",
    "created_provenance": "" ,
    "credit_line": "creditLine",
    "collection" : "Department",
    "current_status" : "",
    "current_owner": "Tate Museum",
    "image_url":"thumbnailUrl",
    "homepage" : "url"
}



for file in files:
    objProp = {}
    with open(fileprefix + file) as json_file:
        data = json.load(json_file)
        objProp =  {
            "id": data["id"],
            "accession_number":data["acno"],
            "accession_date": data["acquisitionYear"],
            "classification" : data["classification"],
            "title": data["title"],
            "alt_title": "",
            "notes": "",
            "date_created":data["dateText"],
            "date_created_earliest": "",
            "date_created_latest": "",
            "created_period":"",
            "created_dynasty":"",
            "created_inscriptions":"",
            "created_notes": "",
            "creator":[],
            "physical_medium": data["medium"],
            "physical_style": "",
            "physical_technique": "",
            "physical_description": "",
            "physical_dimensions": data["dimensions"],
            "created_provenance": "" ,
            "credit_line": data["creditLine"],
            "collection" : "",
            "current_status" : "",
            "current_owner": "Tate Museum",
            "image_url" : "thumbnailUrl",
            "homepage" : "url"
            }
        
        creator = data["contributors"]
        for contributor in creator:
            id = str(contributor["id"])
            name = str(contributor["fc"])
            objProp["creator"].append({"id":id,"name":name})
            
        objProp["current_owner"] = {"name":"Tate Museum",
                                "location":"London,England",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
        id = str(objProp["id"])
        object_uri = baseURI + str(id)
    
        # create obj description
        objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
     
        # write to file 
        text_file = open(outputdir + "/" + id + ".json", "wt")
        n = text_file.write(factory.toString(objLA, compact=False))
        text_file.close()
        

## Harvard Art Museum

https://harvardartmuseums.org/collections/person/28419?person=28419


In [50]:
apidocs = "https://github.com/harvardartmuseums/api-docs"

keyfile = "data/ruskin/input/key.txt"
key = open(keyfile, mode='r', encoding='utf-8-sig').read()
uri = "https://api.harvardartmuseums.org/object?person=28419&apikey=" + key

for page in range(1, 11):
    response = requests.get(uri + "&page=" + str(page))
    json_data = response.json()
    text_file = open( "./data/harvard/input/" + str(page) + ".json", "wt")
    n = text_file.write(json.dumps(json_data, indent=2))
    text_file.close()
    

### mapping

In [51]:
import re

baseURI = "https://www.harvardartmuseums.org/collections/object/"

mapp_harvard =  {
    "id":"id",
    "accession_number":"objectnumber",
    "accession_date": "accessionyear",
    "classification" : "",
    "title": "title",
    "alt_title": "",
    "notes": "commentary",
    "date_created":"dated",
    "date_created_earliest": "",
    "date_created_latest": "",
    "created_period":"period",
    "created_dynasty":"century",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"people",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "technique",
    "physical_description": "description",
    "physical_dimensions": "dimensions",
    "created_provenance": "provenance" ,
    "credit_line": "creditline",
    "collection" : "division",
    "classification": "classification",
    "current_status" : "",
    "current_owner": "Harvard Museum of Art",
    "image_url":"primaryimageurl",
    "homepage" : "url"
}

def createObjPropHarvard(obj,mapp):

    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if key == "dated":
                    
                    # look for year in value 
                    years = re.findall('(\d{4})', obj[key])
                    if len(years) == 1:
                        #objProp["date"] = years[0]
                        objProp["date_created_earliest"] = years[0]
                        objProp["date_created_latest"] = years[0]
                    elif len(years) == 2:
                       # objProp["dated"] = str(years[0]) + "-" + str(years[1])
                        objProp["date_created_earliest"] = years[0]
                        objProp["date_created_latest"] = years[1]
                    else:
                        # use Ruskins birth/death years
                        objProp["date_created_earliest"] = 1819
                        objProp["date_created_latest"] = 1900
                        
    
                        
                        
                
                if prop == "creator":
                    objProp["creator"] = ""
                    for person in obj[key]:
                        
                        if "personid" in person and person["personid"] == 28419 and person["role"] == "Artist": 
                            objProp[prop] = [{
                                "id": str(person["personid"]),
                                "name": person["displayname"],
                                }]      
                else:
                    objProp[prop] = obj[key]
    objProp["current_owner"] = {"name":"Harvard Art Museum",
                                "location":"Cambridge, MA",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    return objProp   

for x in range(1,11):
    
    with open("./data/harvard/input/" + str(x) + ".json") as json_file:
        data = json.load(json_file)
        for obj in data["records"]:
            #print(json.dumps(obj,indent=2))
            objProp=createObjPropHarvard(obj,mapp_harvard)
            print(json.dumps(objProp,indent=2))
            if "image_url" in objProp and objProp["image_url"] != "":
                images["https://www.harvardartmuseums.org/collections/object/" + str(objProp["id"])] = objProp["image_url"]
            if objProp["creator"] != "":
                #print(json.dumps(objProp,indent=2))
                
                id = str(objProp["id"])
                object_uri = baseURI + id
    
                # create obj description
                objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
                #print(json.dumps(objLA,indent=2))
                # write to file 
                text_file = open(outputdir + "/" + id + ".json", "wt")
                #print(factory.toString(objLA, compact=False))
                n = text_file.write(factory.toString(objLA, compact=False))
                text_file.close()
    
    
    



{
  "credit_line": "Harvard Art Museums/Fogg Museum, Transfer from the Fine Arts Department, Harvard University",
  "collection": "European and American Art",
  "id": 93100,
  "created_period": null,
  "accession_date": null,
  "classification": "Prints",
  "image_url": "https://nrs.harvard.edu/urn-3:HUAM:INV195646_dynmc",
  "notes": "Reproduction:  See \"Works of John Ruskin,\" Library edition, edited by E. T. Cook and Alexander Wedderburn, London, Allen 1903, vol iii, opp. p. 338\r\n",
  "physical_technique": "Etching",
  "physical_description": null,
  "physical_medium": null,
  "title": "Foreground Study",
  "created_provenance": null,
  "date_created_earliest": 1819,
  "date_created_latest": 1900,
  "date_created": "19th century",
  "creator": "",
  "homepage": "https://www.harvardartmuseums.org/collections/object/93100",
  "created_dynasty": "19th century",
  "accession_number": "FA5",
  "physical_dimensions": "plate: 253 x 163 cm (99 5/8 x 64 3/16 in.)",
  "current_owner": {
   

{
  "credit_line": "Harvard Art Museums/Fogg Museum, Gift of Miss Mabel Sturgis",
  "collection": "European and American Art",
  "id": 298635,
  "created_period": null,
  "accession_date": 1928,
  "classification": "Drawings",
  "image_url": "https://nrs.harvard.edu/urn-3:HUAM:48817_dynmc",
  "notes": null,
  "physical_technique": null,
  "physical_description": null,
  "physical_medium": "Black ink, watercolor, white gouache, graphite on cream card, darkened to brown",
  "title": "Tower of Fribourg",
  "created_provenance": null,
  "date_created_earliest": "1854",
  "date_created_latest": "1854",
  "date_created": "1854",
  "creator": [
    {
      "id": "28419",
      "name": "John Ruskin"
    }
  ],
  "homepage": "https://www.harvardartmuseums.org/collections/object/298635",
  "created_dynasty": "19th century",
  "accession_number": "1928.12",
  "physical_dimensions": "23.6 x 19.2 cm (9 5/16 x 7 9/16 in.)",
  "current_owner": {
    "name": "Harvard Art Museum",
    "location": "Camb

## Convert JSON file to CSV

In [52]:
fn()

In [56]:
file_list=os.listdir(r"./data/ruskin/output/json/")
  
sources = {   
    "https://www.nga.gov/collection/" : "National Gallery of Art",
    "https://clevelandart.org/art/": "Cleveland Museum of Art",
    "https://www.philamuseum.org/collection/object/": "Philadelphia Museum of Art",
    "https://www.tate.org.uk/art/artworks/" : "Tate Museum",
    "https://www.harvardartmuseums.org/collections/object/" : "Harvard Art Museum"
}  
    
    
json_all = {}
for file in file_list:
    # read file and append to 
    with open( "data/ruskin/output/json/" + file) as json_file:
        json_text = json.load(json_file)
        json_all.update({file : json_text})
        
my_dict = []

title_rows = [
     {
        "Year": "",
        "Month" : "",
        "Day" : "",
        "Time" : "",
        "End Year": "",
        "End Month" : "",
        "End Day" : "",
        "End Time" : "",
        "Display Date": "",
        "Headline" :   "John Ruskin" ,
        "Text": "<p>This timeline visualisation shows artworks created by John Ruskin.</p><p>It demonstrates how the Linked Art data model can be used to transform, reconcile and visualise collections data for artworks.</p> <p>See https://linked.art for more information.</p>",
        "Media": "https://upload.wikimedia.org/wikipedia/commons/0/0a/John_Ruskin_1863.jpg",
        "Media Credit": "Wikipedia",
        "Media Caption": "John Ruskin",
        "Media Thumbnail" : "",
        "Type": "title",
        "Group" :"",
        "Background": ""   },
    
   {
        "Year": "1819",
        "Month" : 2,
        "Day" : 8,
        "Time" : "",
        "End Year": "1900",
        "End Month" : 1,
        "End Day" : 20,
        "End Time" : "",
        "Display Date": "",
        "Headline" :   "John Ruskin's lifetime" ,
        "Text": "",
        "Media": "https://en.wikipedia.org/wiki/John_Ruskin#/media/File:John_Ruskin_1863.jpg",
        "Media Credit": "Wikipedia",
        "Media Caption": "John Ruskin",
        "Media Thumbnail" : "",
        "Type": "era",
        "Group" :"",
        "Background": ""},
    
]

my_dict = title_rows

for file in json_all:
   
    artwork = json_all[file]
    
    
    
    id = artwork["id"]
    #print(json.dumps(artwork,indent=2))
    
    if "produced_by" not in artwork:
        continue
    
    if "begin_of_the_begin" not in artwork["produced_by"]["timespan"]:
        continue
        
    if artwork["produced_by"]["timespan"]["begin_of_the_begin"] == 1819:
        continue
    
    credit = ""
    for source in sources:
        if source in id:
            credit = sources[source]
            
    text = ""
    imageurl = ""
    
    if "referred_to_by" in artwork and len(artwork["referred_to_by"]) > 0 and "content" in artwork["referred_to_by"][0]:  
        text = artwork["referred_to_by"][0]["content"]
      
    homepage = ""
    if artwork["subject_of"][0]["classified_as"][1]["id"] == "http://vocab.getty.edu/aat/300266277":
        homepage = artwork["subject_of"][0]["id"]
        text = text + "<br/><br/><a target='_new' href='" + homepage+ "'>Artwork homepage</a>"
   
        
    if id in images:
        imageurl = images[id]
   
    if "begin_of_the_begin" in artwork["produced_by"]["timespan"]:
        begin = artwork["produced_by"]["timespan"]["begin_of_the_begin"]
    try:
        begin = int(begin)
    except:
        begin = 1819
    
    if "end_of_the_end" in artwork["produced_by"]["timespan"]:
        end = artwork["produced_by"]["timespan"]["end_of_the_end"]
    try:
        end = int(end)
    except:
        end = 1900
    
    row = {
        "Year": begin,
        "Month" : 1,
        "Day" : 1,
        "Time" : "",
        "End Year": end,
        "End Month" : 12,
        "End Day" : 31,
        "End Time" : "",
        "Display Date": artwork["produced_by"]["timespan"]["_label"],
        "Headline" :   artwork["_label"] ,
        "Text": text,
        "Media": imageurl,
        "Media Credit": credit,
        "Media Caption": "",
        "Media Thumbnail" :  imageurl,
        "Type": "title",
        "Group" :"",
        "Background": ""     
    }
    my_dict.append(row)

with open('./data/ruskin/output/csv/ruskin_vis.csv', 'w') as f:  
    w = csv.DictWriter(f, my_dict[0].keys())
    w.writeheader()
    for row in my_dict:
        w.writerow(row)
       

## Visualisation
Follow instructions at https://timeline.knightlab.com/#make to make a timeline using a Google Spreadsheet

In [ ]:
#spreadsheet used as input to timeline visualisation
file="data/ruskin/output/csv/ruskin_vis.csv"

mpg = pd.read_csv(file)
mpg.head()

In [15]:
vis = "https://cdn.knightlab.com/libs/timeline3/latest/embed/index.html?source=1LttNXCohJsD7XByCSt3KVUC02_J57DFOwtb5Cxq8JuI&font=Default&lang=en&initial_zoom=1&height=650"

In [36]:
display(IFrame(vis, width='100%', height=700))

In [17]:
print(vis)

https://cdn.knightlab.com/libs/timeline3/latest/embed/index.html?source=1LttNXCohJsD7XByCSt3KVUC02_J57DFOwtb5Cxq8JuI&font=Default&lang=en&initial_zoom=1&height=650
